In [1]:
from pyspark.sql import *
from pyspark.sql.functions import * 

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
spark

In [4]:
players_df = spark.read.csv("/user/itv007180/winning_streak.txt",header=True,inferSchema=True)

In [5]:
players_df.show()

+---------+----------+------+
|player_id|match_date|result|
+---------+----------+------+
|        1|2023-10-17|   Win|
|        1|2023-10-18|   Win|
|        1|2023-10-20|  Draw|
|        1|2023-10-23|   Win|
|        1|2023-10-25|   Win|
|        1|2023-10-31|   Win|
|        1|2023-11-01|   Win|
|        1|2023-11-02|   Win|
|        2|2023-10-20|  Draw|
|        2|2023-10-22|  Lost|
|        2|2023-10-27|   Win|
|        3|2023-11-01|  Lost|
+---------+----------+------+



In [6]:
mywindow = Window().partitionBy("player_id").orderBy("match_date")

modified_df = players_df.withColumn("day1_result",lead(players_df.result,1,'NULL').over(mywindow)) \
.withColumn("day2_result",lead(players_df.result,2,'NULL').over(mywindow))

In [7]:
modified_df.show()

+---------+----------+------+-----------+-----------+
|player_id|match_date|result|day1_result|day2_result|
+---------+----------+------+-----------+-----------+
|        1|2023-10-17|   Win|        Win|       Draw|
|        1|2023-10-18|   Win|       Draw|        Win|
|        1|2023-10-20|  Draw|        Win|        Win|
|        1|2023-10-23|   Win|        Win|        Win|
|        1|2023-10-25|   Win|        Win|        Win|
|        1|2023-10-31|   Win|        Win|        Win|
|        1|2023-11-01|   Win|        Win|       NULL|
|        1|2023-11-02|   Win|       NULL|       NULL|
|        3|2023-11-01|  Lost|       NULL|       NULL|
|        2|2023-10-20|  Draw|       Lost|        Win|
|        2|2023-10-22|  Lost|        Win|       NULL|
|        2|2023-10-27|   Win|       NULL|       NULL|
+---------+----------+------+-----------+-----------+



In [8]:
from pyspark.sql.functions import expr

In [9]:
##final_df = modified_df.withColumn("count1",expr("CASE WHEN result = 'Win' THEN 1")).show()

In [17]:
final_df = modified_df.filter("result = 'Win'" and ( "day1_result = 'Win'" or "day1_result = 'NULL'" ) and ("day2_result = 'Win'" or "day2_result = 'NULL'"))

In [19]:
final_df.groupBy("player_id").agg(count("player_id")).show()

+---------+----------------+
|player_id|count(player_id)|
+---------+----------------+
|        1|               5|
|        2|               1|
+---------+----------------+



In [31]:
#modified_df.withColumn("count1",expr("CASE WHEN result = 'Win' and (day1_result = 'Win' or day1_result = 'NULL') and (day2_result = 'Win' or day2_result = 'NULL') THEN 1"))

ParseException: 
mismatched input 'result' expecting {<EOF>, '-'}(line 1, pos 10)

== SQL ==
CASE WHEN result = 'Win' and (day1_result = 'Win' or day1_result = 'NULL') and (day2_result = 'Win' or day2_result = 'NULL') THEN 1
----------^^^
